In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.7451, 0.3092, 0.3084],
        [0.8476, 0.0872, 0.4444],
        [0.2314, 0.7422, 0.4812],
        [0.2404, 0.2411, 0.3269],
        [0.7614, 0.4478, 0.2744]])


In [2]:
import torch

if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("GPU model:", torch.cuda.get_device_name(0))
else:
    print("No GPU available")

No GPU available


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from trl import SFTConfig
hf_token = os.getenv('HF_TOKEN')

/Users/tristanherink/cs263_final_project/263-project/cs263_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# The model that you want to train from the Hugging Face hub
model_name = '/Users/tristanherink/cs263_final_project/263-project/Meta-Llama-3-8B-Instruct'


################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 4096

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
###device_map='auto'

In [5]:
# Load dataset (you can process it here) 
dataset = load_dataset("csv", data_files='/Users/tristanherink/cs263_final_project/263-project/test_llama.csv')

Generating train split: 23 examples [00:00, 1651.84 examples/s]


In [6]:
#sample_dataset = dataset.shuffle().select(range(10000))

In [7]:
#sample_dataset

In [8]:
#print(sample_dataset['text'][0])

In [6]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

 

# Load base model
model = AutoModelForCausalLM.from_pretrained(model_name,load_in_8bit=False)
'''
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config
    #device_map=device_map,token=hf_token
)
'''
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 4/4 [00:31<00:00,  7.80s/it]


In [7]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,token=hf_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Must add EOS_TOKEN at response last line
tokenizer.pad_token = tokenizer.eos_token
EOS_TOKEN = tokenizer.eos_token
def concatnate(example):
    concatenated_str = (
        f"The class name is {example['class_name']}. "
        f"The class dates are {example['class_dates']}. "
        f"The class times are {example['class_times']}. "
        f"The class year is {example['year']}. "
        f"The quarter is {example['semester']}."
    )
    return {"concatenated_text": concatenated_str} 
def prompt_eos(sample):
    sample['concatenated_text'] = sample['concatenated_text']+EOS_TOKEN
    return sample
dataset = dataset.map(concatnate)
dataset = dataset.map(prompt_eos)

Map: 100%|██████████| 23/23 [00:00<00:00, 6094.06 examples/s]


In [9]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['class_name', 'class_dates', 'class_times', 'year', 'semester', 'concatenated_text'],
        num_rows: 23
    })
})

In [13]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer( #SFTConfig
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    dataset_text_field="concatenated_text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/Users/tristanherink/cs263_final_project/263-project/cs263_venv/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
Map: 100%|██████████| 23/23 [00:00<00:00, 1009.89 examples/s]


RuntimeError: MPS backend out of memory (MPS allocated: 20.29 GB, other allocations: 1.72 MB, max allowed: 20.40 GB). Tried to allocate 224.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [22]:
trainer.train()


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



Step,Training Loss


KeyboardInterrupt: 

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistance who help ucla students to choose their classes"},
    {"role": "user", "content": "what the class name is"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True
).to(model.device)

In [24]:
messages = [
    {"role": "system", "content": "You are a helpful assistance who help ucla students to choose their classes"},
    {"role": "user", "content": "what the class name is"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=255,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I'd be happy to help you find a class! What kind of class are you looking for? Do you have a specific subject in mind, or are you open to suggestions? Also, what's your major or area of interest? That might help me give you more targeted suggestions.

If you need some inspiration, here are some popular class categories at UCLA:

* Arts and Humanities
* Social Sciences
* Life Sciences
* Physical Sciences
* Engineering and Computer Science
* Law
* Medicine, Dentistry, and Public Health
* Public Affairs
* Education
* Agriculture, Environment, and Life Sciences
* Architecture and Allied Arts
* Fine and Performing Arts

Let me know if any of these categories interest you, or if you have something else in mind!
